In [1]:
import numpy as np
import keras

from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Dropout, Activation,LSTM,TimeDistributed
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction import image
from keras.utils.np_utils import to_categorical
%matplotlib inline

Using TensorFlow backend.
/home/retief/anaconda3/envs/envSKA3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
#Settings
max_words = 1000
batch_size = 150
epochs = 5

In [ ]:
#Loading data
print('Loading data...')
data_text = np.load('./data/SAXED_TEST_MP_PXX_20160302_000000_IMAGE_25_days.npy')
data_label = np.load('./data/SAXED_TEST_MP_PXX_20160302_00000_MASK_25_days.npy')

# #convert to matrix of shape (276,1000)
# data_text = data_text.flatten().astype(str);
# np.shape(data_label)

Loading data...


In [ ]:

patches_data = image.extract_patches_2d(np.array(data_text)[:,0:10000], (276,1))
patches_label = image.extract_patches_2d(np.array(data_label)[:,0:10000], (276,1))
np.shape(patches_data)

In [ ]:
v_label = np.array( [sum(u)/276 for u in patches_label]).flatten()
v_label[v_label >= 0.75] = 1
v_label[v_label < 0.75] = 0

In [ ]:
# v_label = to_categorical(v_label)

In [ ]:
plt.hist(v_label)

In [ ]:
#Split in data and labels into testing and training
x_train, x_test, y_train, y_test = train_test_split(patches_data, v_label, test_size=0.30, random_state=0)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')


In [ ]:
length = 1
n_neurons = length
n_batch = 10
n_epoch = 1

In [ ]:
print('Building model...')
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(patches_data.shape[1], patches_data.shape[2])))
model.add(Dense(1))
model.add(Dropout(0.7))
model.add(Activation('sigmoid'))



In [ ]:

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=False)
model.compile(
              loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1
                   )
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
prediction = model.predict(x_train, verbose=1)

In [ ]:
prediction[prediction >= 0.75] = 1
prediction[prediction < 0.75] = 0

uniques, ids = np.unique(prediction, return_inverse=True)
prediction= uniques[prediction.argmax(1)]

In [ ]:
data_image = np.load('./data/SAXED_TEST_MP_PXX_20160302_000000_IMAGE_25_days.npy')
fig1 = plt.figure(figsize=(15,15))
plt.subplot(2,1,1)
plt.imshow(np.array(data_image)[:,:])
plt.subplot(2,1,2)
plt.plot(prediction[0:])
plt.ylim(0,1)